* Alice, Bob, and Charlie are the candidates for accepting values from each participant.
* Values from each user are randomly split among the candidates.
* The split value is encrypted using Paillier encryption and sent to the candidates using the candidates' public key.
* After receiving all the values from participants, the candidates will perform Additive Homomorphic encryption of all the encrypted ciphertext.
* The resulting ciphertext is decrypted at the candidate's side using the private key of the candidate.
* The sum of decrypted values among the candidates is the total sum sent by all the participants.


In [7]:
#Generic Methods
import random

def gcd_extended(a, b):
    if a == 0:
        return b, 0, 1

    gcd, x1, y1 = gcd_extended(b % a, a)

    x = y1 - (b // a) * x1
    y = x1

    return gcd, x, y

def modular_inverse(a, n):
    gcd, x, _ = gcd_extended(a, n)
    if gcd == 1:
        return x % n
    return None

def generate_prime(bits):
    while True:
        num = random.getrandbits(bits)
        if num % 2 != 0 and is_prime(num):
            return num

def is_prime(n, k=40):
    if n <= 3:
        return n == 2 or n == 3

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)

        if x == 1 or x == n - 1:
            continue

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def distribute_number_as_3(number):
    # Generate random non-zero values that sum up to 'number - 1'
    half = number // 2
    values = random.sample(range(1, half + 1), 2)
    values.append(number - sum(values))
    new_tuple = tuple(int(item) for item in values)

    return tuple(new_tuple)

import random

def distribute_number(number, n):
    distributed_numbers = []

    for _ in range(n-1):
        # Generate a random value between 1 and the remaining number (inclusive)
        value = random.randint(1, number - (n - len(distributed_numbers) - 1))

        # Subtract the generated value from the remaining number
        number -= value

        # Append the generated value to the list
        distributed_numbers.append(int(value))

    # Append the remaining number
    distributed_numbers.append(int(number))
    #distributed_numbers = tuple(int(item) for item in distributed_numbers)
    random.shuffle(distributed_numbers)
    return distributed_numbers

# Example usage
number_to_distribute = 100
resultDistibuted = distribute_number(number_to_distribute,5)
print(resultDistibuted)
type(resultDistibuted[2])

[53, 7, 7, 1, 32]


int

In [8]:
#Paillier Encryption
class PaillierEncryption:
    def __init__(self, key_length=1024):
        self.key_length = 33
        #self.n_squared=0
        #self.key_length = key_length
        self.public_key, self.private_key = self.generate_key_pair()
        self.container=[]

    def generate_key_pair(self):
        p = generate_prime(self.key_length // 2)
        q = generate_prime(self.key_length // 2)

        n = p * q
        n_squared = n ** 2
        g = n + 1
        l = (p - 1) * (q - 1)
        mu = modular_inverse(l, n)

        public_key = (n, g)
        private_key = (l, mu)

        return public_key, private_key

    def encrypt(self, plaintext):
        n, g = self.public_key
        r = random.randint(1, n - 1)
        n_squared = n ** 2
        #print(g,plaintext,n_squared,r,n)
        ciphertext = (pow(g, plaintext, n_squared) * pow(r, n, n_squared)) % n_squared
        return ciphertext

    def decrypt(self, ciphertext):
        n, _ = self.public_key
        l, mu = self.private_key
        n_squared = n ** 2

        numerator = pow(ciphertext, l, n_squared) - 1
        plaintext = (numerator // n) * mu % n
        return plaintext
    
    def mult_paillier(self , m, pc):
        n, g = self.public_key
        n_squared = n ** 2
        mult= pow(pc, m, n_squared)
        return mult
    
    def add_paillier(self, pc1 , pc2):
        n, _ = self.public_key
        n_squared = n ** 2
        res=(pc1*pc2 ) % n_squared
        return res
    def add_to_queue(self,value):
        self.container.append(value)
    def Retriev_Container(self):
        return self.container
    def multiply_Container(self):
        multVal=1
        for i in range(len(self.container)):
            #print(self.container[i])
            multVal=multVal*self.container[i]
        return multVal
    


In [9]:
#Testing Encryption and decryption
plaintext = 333
paillier = PaillierEncryption()
ciphertext = paillier.encrypt(plaintext)
decrypted_plaintext = paillier.decrypt(ciphertext)

print("Plaintext:", plaintext)
print("Ciphertext:", ciphertext)
print("Decrypted plaintext:", decrypted_plaintext)


Plaintext: 333
Ciphertext: 351502632955699963
Decrypted plaintext: 333


In [10]:
#Additive homomorphic property of Paillier
m1=10
m1C = paillier.encrypt(m1)
m2=20
m2C = paillier.encrypt(m2)
# m1m2C=m1C*m2C
# print(m1C,m2C,m1m2C)
# paillier.decrypt(m1m2C)
m1m2C=paillier.add_paillier(m1C,m2C)
paillier.decrypt(m1m2C)

30

In [11]:
#Multiplicative homomorphic property of Paillier
m1=5
m1C = paillier.encrypt(m1)
n=2
m1pown=m1C**n
print(m1C, n,m1pown)
paillier.decrypt(m1pown)

148861840481775396 2 22159847551621544062684812190956816


10

In [23]:
#Privacy Preserved Per Capita Saving Calculation
import numpy as np

participantsCount=10
candidateCount=3
    
AliceP = PaillierEncryption()
BobP = PaillierEncryption()
ChalieP = PaillierEncryption()
candidates=[AliceP,BobP,ChalieP]
CandidateNames=["Alice","Bob","Charlie"]
InputValues=np.array([100,200,150,80,90,180])
print("Input Values Of All Participants: ",InputValues)

for gdpInddex in range(len(InputValues)):
    ValueToProcess=InputValues[gdpInddex]
    print("\nParticipant",gdpInddex+1, "Value To Distribute" ,":",ValueToProcess)
    dividedNumber=distribute_number(ValueToProcess,candidateCount)

    for divideIndex in range(len(dividedNumber)):
        passingValue=dividedNumber[divideIndex]
        candidateP=candidates[divideIndex]
        print("Passing Encrypted Value" , passingValue ," to ", CandidateNames[divideIndex])
        passingValueC=candidateP.encrypt(passingValue) 
        candidateP.add_to_queue(passingValueC)

AliceSumC=AliceP.multiply_Container()
AliceSumDecrypt= AliceP.decrypt(AliceSumC)
print("\nSum of Values Decrypted By Alice : ",AliceSumDecrypt)

BobSumC=BobP.multiply_Container()
BobSumDecrypt=BobP.decrypt(BobSumC)
print("Sum of Values Decrypted By Bob : ",BobSumDecrypt)

ChalieSumC=ChalieP.multiply_Container()
ChalieSumDecrypt=ChalieP.decrypt(ChalieSumC)
print("Sum of Values Decrypted By Chalie : ",ChalieSumDecrypt)

sumOfDerypt=AliceSumDecrypt+BobSumDecrypt+ChalieSumDecrypt
print("\nExpected Sum Of All Participant Values: ",sum(InputValues))
print("Actual Sum Of Decrypted Values By All Candidates: ",sumOfDerypt)

Input Values Of All Participants:  [100 200 150  80  90 180]

Participant 1 Value To Distribute : 100
Passing Encrypted Value 13  to  Alice
Passing Encrypted Value 82  to  Bob
Passing Encrypted Value 5  to  Charlie

Participant 2 Value To Distribute : 200
Passing Encrypted Value 76  to  Alice
Passing Encrypted Value 40  to  Bob
Passing Encrypted Value 84  to  Charlie

Participant 3 Value To Distribute : 150
Passing Encrypted Value 22  to  Alice
Passing Encrypted Value 24  to  Bob
Passing Encrypted Value 104  to  Charlie

Participant 4 Value To Distribute : 80
Passing Encrypted Value 50  to  Alice
Passing Encrypted Value 26  to  Bob
Passing Encrypted Value 4  to  Charlie

Participant 5 Value To Distribute : 90
Passing Encrypted Value 71  to  Alice
Passing Encrypted Value 9  to  Bob
Passing Encrypted Value 10  to  Charlie

Participant 6 Value To Distribute : 180
Passing Encrypted Value 21  to  Alice
Passing Encrypted Value 148  to  Bob
Passing Encrypted Value 11  to  Charlie

Sum of Valu